In [1]:
%load_ext autoreload
%autoreload 2

from src.settings import Paths, Settings
from src.data_loader import CLEARDataLoader
from ieeg_data_loader.data import iEEGDataLoader
import pandas as pd



In [2]:
settings = Settings()  # Initialize settings object
settings.load_settings()  # Load settings from a JSON file
settings.dataset = 'clear'
settings.dataset_task = 'flicker'
settings.patient = 'p05' # [f'p{i+1:02}' for i in range(18)]
print(f"Patients: {settings.patient}")

In [3]:
paths = Paths(settings)  # Initialize paths object with loaded settings
paths.load_device_paths()  # Load device-specific paths
paths.create_paths()  # Create any necessary file paths

In [4]:
features_raw_df_list = []
meta_data_all = []
for patient in settings.patient:
    # Load EEG dataset using configured settings and paths
    dataset = CLEARDataLoader(paths=paths, settings=settings)
    dataset.load_data(patient_ids=patient)  # Load EEG data for specified patients
    data_loader = iEEGDataLoader(patient=patient,
                                     target_class='color',
                                     prepared_dataset_path=paths.raw_dataset_path,
                                     task=settings.dataset_task,
                                     file_format='npy')
    dataset_list = data_loader.load_data()
    
    for dataset in dataset_list:
        meta_data = {
            'Patient': patient,
            'Block': dataset.meta[-1],
            'Task': settings.dataset_task,
            'Num_Channels': len(dataset.channel_name),
            'Num_Trials': dataset.data.shape[0]
        }
        meta_data_all.append(meta_data)
        


In [5]:
df_info = pd.DataFrame(meta_data_all)
df_info.to_csv(paths.eda_results + 'm_sequence_task_summary.csv')